<a href="https://colab.research.google.com/github/suprabhatkr/machine_learning_projects/blob/master/sequicity/Sequicity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd '/content/drive/My Drive/ML/sequicity-master/'

/content/drive/My Drive/ML/sequicity-master


In [ ]:
import keras
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,LSTM,Input,Bidirectional
from sklearn.model_selection import cross_val_score 
from nltk.tokenize import word_tokenize,sent_tokenize
import nltk
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
def bleu_score(reference,candidate):
  reference=[word_tokenize(reference)] 
  candidate=word_tokenize(candidate)
  score = sentence_bleu(reference, candidate)
  return score

In [ ]:
def isint(word):
  try:
    word=int(word) 
  except:
    word=word 
  if type(word)==int and int(word)==float(word):
    return 1 
  else:
    return 0
def isNan(word):
    try:
        word=int(word) 
    except:
        word=0 
    return word
def isfloat(word):
  try:
    word=float(word) 
  except:
    word=word 
  if type(word)==int:
    return 1 
  else:
    return 0 

def get_pos(arr):
  ans=[]
  for i in range(len(arr)):
    if arr[i]>0.08:
      ans.append(i) 
  return ans
nltk.download('punkt')
def padd(string):
  arr=word_tokenize(string)  
  l=len(arr) 
  for i in range(max(0,15-l)):
    arr.append('<pad>') 
  return arr[:15]  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
clean_mp={'6gb': '6 gb', '34gb': '34 gb', '128gb': '128 gb', '3gb': '3 gb','16-megapixel':'16 mp','with15mp': 'with 15 mp', '10mp': '10 mp', '8mp': '8 mp', 'mp-primary': 'mp for primary', 'mp-secondary': 'mp for secondary', '13mp': '13 mp', '16mp': '16 mp', '23mp': '23 mp', '20mp': '20 mp', '12mp': '12 mp','4gb':'4 gb','dualsim':'dual sim','feauture':'feature','gread':'great','..':'.','resposnse':'response','dscount':'discount','ofcourse':'of course','non-smartphone':'non smartphone','freindly':'friendly','perfec':'perfect','rs9999':'rs 9999','asssist':'assist','9000/-':'9000','speciific':'specific','phonebecause':'phone because','188gms':'188 gms','168gms':'168 gms','possesor':'processor','ramand':'ram and','youngeters':'youngsters','multi-':'multi','than32':'than 32','ssure':'sure','5megapixel':'5 mp','addtional':'additional','widely-adopted':'widely adopted','speciifc':'specific','avilable':'available','appliactions':'appliacations','xiomi':'xiaomi','multisim':'multi sim','applehas':'apple has','buawei':'huawei','specificctions':'specifications','non-smartphones':'non smartphones','spcific':'specific','itune':'itunes','brandyou':'brand you','goood':'good','awsome':'awesome','dual-primary':'dual primary','selfie':'photo','multitaking':'multitasking','widely-adopted':'widely adopted','gioned':'gionee','coolp':'coolpad','adbrand':'brand','advanatge':'advantage','ecpecially':'especially','movies/netflix':'movies and netflix','eqipped':'equipped','samsungmobile':'samsung mobile','samusng':'samsung','260euro':'260 euro', '750euro':'750 euro', '250euro':'250 euro', '550euro':'550 euro', '750euro':'750 euro'}
val_mp={'good': 'good', 'black': 'black', 'android': 'and', 'y': 'yes', 'white': 'white', 'grey': 'grey', 'less': 'less', 'high': 'high', '15k': '15k', '10k': '10k', 'latest': 'latest', 'non': 'none', 'no': 'no', 'vivo': 'vivo', 'i': 'i', 'gold': 'gold', '15+': '15', 'windows': 'windows', '15k+': '15k', 'nokia': 'nokia', 'e': 'e', '16k+': '16k', '20k': '20k', '2019': '2019', '2k': '2k', '6gb': '6gb', '15': '15', 'jio': 'jio', 'f': 'f', 'blue': 'blue', '4+': '4', 'silver': 'silver', 'big': 'big', 'light': 'light', 'htc': 'htc', 'paradise': 'paradise', 'lenovo': 'lenovo', 'yoga tab 3': 'yoga tab 3', 'yes': 'yes', '400': '400', ' light': 'light', '64gb': '64gb', 'd': 'dual', 't': 't-mobile', 'non black': 'non-black', 'small': 'small', 'non and': 'ios', 'samsung': 'samsung', 'samsung ': 'samsung', 'expensive': 'expensive', '128': '128', 'non silver': 'non-silver', '16+': '16', '2016': '2016', '500': '500', '2017': '2017', '128 gb': '128gb', '800+': '800', 'apple': 'apple', '700': '700', '20': '20', 'and': 'and', 'black&blue': 'black', '650': '650', '64+': '64', 'may': 'may', '4000+': '4000', 'pink': 'pink', 'good  ': 'good', '400+': '400', '64': '64', 'non gunmetal': 'non-gunmetal', 'sony': 'sony', 'xperia xa ultra': 'xperia xa ultra', 'non oppo': 'vivo', 'lg': 'lg', 'acer': 'acer', 'iconia tab a700 ': 'iconia tab a700', 'non grey': 'non-grey', 'oppo': 'oppo', '350': '350', 'non crown gold': 'non-gold', 'pearl white\n': 'white', 'recent': 'recent', '300': '300', '20+': '20', '32+': '32', 'blue||gold': 'blue||gold', '8000+': '8000', 'motorola': 'motorola', '150': '150', '12+': '12', '6': '6', '3': '3', '32': '32', 'galaxy note7': 'galaxy note7', '3000+': '3000', 'null': 'null', '13+': '13', '250': '250', 'aquamarine green': 'green', 's': 's', '200': '200', 'pearl white': 'white', 'huawei': 'huawei', 'u8150 ideos': 'u8150 ideos', 'blackberry': 'blackberry', '6000+': '6000', 'good      ': 'good', 'm': 'm', '16': '16', 'onyx black': 'black', 'green': 'green', 'heavy': 'heavy', 'bla': 'bla', ' 300': '300', '220': '220', 'sym': 'sym', 'iconia tab a501\n': 'iconia tab a501', ' non and': 'ios', 'mic': 'mic', 'non dual': 'non-dual', 'titanium grey': 'grey', 'xiaomi': 'xiaomi', '800': '800', 'ios': 'ios', '64 gb': '64gb', '16 ': '16', '': '', 'chrome': 'chrome', 'gray': 'gray', 'non blue': 'non-blue', 'sony ericsson': 'ericsson', 'xperia arc s': 'xperia', '3500+': '3500', 'good     ': 'good', '10+': '10', 'asus': 'asus', '450': '450', '340': '340', 'leeco\n': 'leeco', '160': '160', 'medium': 'medium', 'good        ': 'good', '2014': '2014', 'non gold': 'non-gold', '600': '600', '4': '4', '2015': '2015', '270': '270', '4.65 hd': 'hd', ' 250': ' 250', 'average': 'average', 'good       ': 'good', 'dual': 'dual', ' and': 'and', 'non vga': 'non-vga', 'orange': 'orange', 'xiamo': 'xiaomi', '1800+': '1800', 'gionee': 'gionee', '5': '5', '140': '140', 'non coolpad': 'non-coolpad', 'alcatel': 'alcatel', 'allview': 'allview', 'zte': 'zte', 'xiaomo ': 'xiaomi', '500+': '500', 'moderate': 'moderate', 'coolpad': 'coolpad', 'gigabyte': 'gigabyte', '2+': '2', 'tiz': 'tiz', 'micromax': 'micromax', '13': '13', 'non white': 'non-white', '280': '280', '3000': '3000', 'vodafone': 'vodafone', 'charcoal black\n': 'black', 'oneplus': 'oneplus', 'vertu': 'vertu', '8': '8', '260': '260', '8+': '8', ' bla': ' bla', '3+': '3', 'meizu': 'meizu', 'archos': 'archos', 'google': 'google', '4500+': '4500', 'hp': 'hp'} 
intent_mp={'Phn_SelReq': 'selling', 'Booking_Req': 'booking'}
u_action_mp={'null':'null','Phn_SelReq': 'phone', 'Rec_Slt': 'rec_slt', 'Cost_Q': 'cost', 'Color_Q': 'color', 'Feedback_Slt': 'feedback', 'Feedback': 'feedback', 'Booking_Req': 'booking', 'Thanks': 'thanks', 'DualSim_Q': 'dualsim', 'Released_Yr_Q': 'year', 'OS_Q': 'os', 'Camera_Q': 'camera', 'Rating_Q': 'rating', 'Confirm_Q': 'confirm', 'Battery_Q': 'battery', 'Memory_Q': 'memory', 'DuslSim_Q': 'dualsim', 'Review_Q': 'review', 'Specification_Q': 'specification', 'Discount_Q': 'discount', 'Weight_g_Q': 'weight', 'Display_Size_Q': 'size', 'Radio_Q': 'radio', 'Internal_RAM_Q,RAM_Q': 'ram', 'P_Camera_Q': 'camera', 'Internal_RAM_Q': 'ram', 'Greet': 'greet', 'P_Camera_Q,S_Camera_Q': 'camera', 'Confirm': 'confirm', 'Released_Yr_Q,Released_Month_Q': 'month', 'RAM_Q,Internal_RAM_Q': 'ram', 'Phn_Info': 'phone', 'S_Camera_Q': 'camera', 'Sim_Q': 'sim', 'RAM_Q': 'ram', 'GPS_Q': 'gps', 'P_Camera,S_Camera_Q': 'camera', 'RAM,_Q,Internal_RAM_Q': 'ram', 'SIM_Q': 'sim'}
a_action_mp={'Specification_D': 'specification', 'Result': 'result', 'Cost_A': 'cost', 'Color_A ': 'color', 'Rephrase': 'rephrase', 'Success': 'success', 'Welcome': 'welcome', 'Color_A': 'color', 'DualSim_A': 'dualsim', 'Released_Yr_A': 'year', 'OS_A': 'os', 'Camera_A': 'camera', 'Rating_A': 'rating', 'Confirm': 'confirm', 'Battery_A': 'battery', 'Memory': 'memory', 'Review_A': 'review', 'Specification_A': 'specification', 'Discount_A': 'discount', 'Confirm_Q': 'confirm', 'Feedback': 'feedback', 'Feedback_R': 'feedback', 'Weight_g_A': 'weight', 'Display_Size_A': 'size', 'Radio_A': 'radio', 'Internal_RAM_A': 'ram', 'RAM_A': 'ram', 'P_Camera_A': 'camera', 'S_Camera_A': 'camera', 'ReleasedYear_A': 'year', 'ReleasedMonth_A': 'month', 'RAM_Q': 'ram', 'Greet': 'greet', 'P_Camera_Q': 'camera', 'Sim_A': 'sim', 'Cost_Q': 'cost', 'GPS_A': 'gps', 'Released_Month_A': 'month', 'Weight_g_Q': 'weight', 'P_Camera': 'camera', 'null': 'null', 'SIM_A': 'sim','Memory_A':'memory'} 
correct={'Camera': 'camera', 'Battery': 'battery', 'Color': 'color', 'OS': 'os', 'Dualsim': 'dual', 'Touch': 'touch', 'Memory': 'memory', 'Cost': 'cost', 'Rating': 'rating', 'Price': 'price', 'Released_Yr': 'year', 'Company': 'company', 'Model': 'model', 'Total_Review': 'review', 'DualSim': 'dual', 'RAM': 'ram', 'Display_Size': 'size', 'Weight_g': 'weight', 'Brand': 'brand', 'Radio': 'radio', 'Internal_RAM': 'ram', 'Sim': 'sim', 'P_Camera': 'camera', 'S_Camera': 'camera', 'Internal_Ram': 'ram', 'GPS': 'gps', 'Internal RAM': 'ram', 'Released_ Yr': 'year', 'Released_Month': 'month', 'SIM': 'sim', 'weight_g': 'weight', 'Weight_G': 'weight', 'Displa_Size': 'size', ' Internal_RAM': 'ram', 'nll': 'nll', 'battery': 'battery', 'Dsplay_Size': 'size', ' Model': 'model', 'Weight': 'weight', 'Apple': 'apple', ' P_Camera': 'camera'}

In [ ]:
mem_arr=['4','6']
year_arr=['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
month_arr=['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
rating_arr=['4 . 1','4 . 2','4 . 3','4 . 4','4 . 5','4 . 6','4 . 7','4 . 8','4 . 9','5 . 0','1','2','3','4','5'] 
bat_arr=['3800', '41000', '38000', '3000', '2400', '3300', '24', '3260', '1785', '2870', '2700', '7300', '32', '2900', '4000', '2620', '2850', '2300', '4010', '2950', '2500', '6020', '3505', '2730', '5000', '304', '2770', '4100', '1830', '2915', '2650', '1200', '2000', '4550', '2100', '1750', '400', '1960', '2050', '2800', '2600', '2530', '2750', '1950', '2110', '2910', '3020', '2760', '4800']
cam_arr=['13', '15', '10', '20', '16', '12', '23', '7', '5', '19', '8', '21', '4', '22', '64', '2', '1']
os_arr=['android','and','ios','mic','windows','blackberry','symphony','mm','coloros','fir']
color_arr=['black', 'mystic black', 'pure white', 'various', 'aquamarine green', 'graphite black', 'titan black', 'titanium black', 'silver', 'white', 'essential white', 'rock black', 'gentle black', 'titanium gray', 'blue', 'soft-touch black', 'satin gold', 'venus gold', 'halo gold', 'gold', 'volcanic grey', 'slate', 'yellow', 'midnight blue', 'lemon yellow', 'dark gray', 'full white', 'bluish black', 'mozilla orange', 'cloudy', 'red', 'green', 'cranberry pink', 'blue', 'flash red', 'twilight blue', 'reddish black', 'ardesia', 'dark blue', 'space gray', 'jet black', 'black(8', 'shimmer gold', 'navy black', 'glacier grey', 'glacier gray', 'gray', 'titanium grey', 'sand gold', 'sapphire black', 'glacier silver', 'charcoal black', 'rosa', 'pure black', 'rose gold', 'osmium black', 'meteorite black', 'burgundy red', 'sapphire black', 'crystal white', 'amethyst gray', 'royal blue', 'grey', 'neon orange', 'midnight black', 'mineral black', 'forest', 'champagne white', 'royal gold', 'champagne gold', 'brown', 'champagne gold', 'glossy black', 'cherry gold', 'jazz gold', 'mocha gold', 'rose gold', 'pearl white', 'quite black', 'amazing silver', 'brilliant black', 'stone black', 'black gold', 'gunmetal gray', 'carbon gray', 'dazzling gold', 'meteor gray', 'meteor grey', 'marble white', 'white birch', 'glacier blue', 'meridian gray', 'indigo black', 'santorini white', 'polar white', 'vanilla white', 'glacial silver', 'violet', 'radical yellow', 'stealth black', 'california blue', 'domino', 'active black', 'clearwater', 'active white', 'brown (teflon coating)', 'sophisticated black', 'yes', 'metal grey', 'amber gold', 'aurora blue', 'ceramic white', 'mystic silver', 'golden black', 'haze gold', 'graphite black', 'agate red', 'pink', 'prestige gold', 'obsidian black', 'moonlight silver', 'aluminium', 'luxurious gold', 'moonlight silver', 'obsidian black', 'snow white', 'crystal black', 'metallic black', 'crystal blue', 'matt black', 'shadow black', 'black body', 'orange', 'deep blue', 'icy white', 'dark blue', 'dark grey', 'metallic brown', 'aurora black', 'onyx black', 'gunmetal grey', 'matte black', 'platinum', 'matte white', 'tuxedo black', 'crimson', 'ebony', 'golden', 'titanium', 'flamingo red', 'mystic white', 'titan', 'indigo', 'titan silver', 'metallic black', 'space black', 'carbon', 'brilliant bronze', 'navy', 'blue', 'metallic gray', 'metallic silver', 'platinum silver', 'silk white', 'blue (boost mobile)', 'titan silver', 'purple', 'black & silver', 'champanage gold', 'mint green', 'black (front panel)', 'black front & white back', 'black (front)', 'white champagne', 'magnetic black', 'classy silver', 'cocoa brown', 'matte cyan', 'glossy cyan', 'iron gray', 'iron grey', 'licorice black', 'lunar gray', 'metallic cherry', 'lunar grey', 'black ballistic nylon', 'deep sea blue', 'black lens', 'champagne', 'licorice', 'cloud gray', 'electric magic black', 'golden bronze', 'black slate', 'midnight titanium', 'arte black', 'bright orange', 'glossy green', 'bright orange', 'bright green', 'cyan', 'lime green', 'matte black', 'steel dark', 'cool grey', 'nokia oro light', 'dark steel', 'frosty metal', 'silver grey', 'charcoal black', 'azure', 'carbon black', 'blue on white', 'chrome', 'white steel', 'black and cyan', 'gunmetal', 'graphite', 'onyx', 'sandstone black', 'obsidian', 'marine blue', 'steel blue', 'black sky', 'blue coral', 'camo green', 'black onyx', 'black sapphire', 'white frost', 'smoky titanium', 'white pearl', 'frosted white', 'titanium green', 'iris charcoal', 'electric blue', 'dazzling white', 'pebble blue', 'cream white', 'dive blue', 'black and gold', 'dark brown', 'luxury brown', 'urban grey', 'metallic blue', 'chic white', 'metallic gray and white', 'navy blue', 'carbon blue', 'noble black', 'metallic gray', 'black and grey', 'deep black', 'black and bronze', 'modern black', 'hanayome white', 'snowy white', 'luminous chrome', 'mineral black', 'serene black', 'classic black', 'blue gradient', 'iconic black', 'sensous black', 'elegant black', 'solid black', 'dark ale', 'sunburst', 'dark gray metallic', 'jet calf', 'onyx calf', 'jet', 'crown gold', 'ceramic black', 'coffee brown', 'moonstone grey', 'ion gold']
brand_arr=['xioami','apple','vodafone','iphone','redmi','zenfone','oppo','vivo','nokia','samsung','allview','htc','gionee','oneplus','accer','acer','sonim','qmobile','panasonic','verykool','huawei','asus','lenovo','coolpad','alcatelis','jio']
model_arr=['model aa', 'model b', 'model ff', 'model d', 'sure , model a', 'model h', 'model j', 'model ff', 'model j', 'model e', 'model k', 'model h', 'model h', 'model e', 'model b', 'model ff', 'model a', 'model ff', 'xplay6', 'iphone 7 plus', 'desire 10 pro', 'zuk z2 pro', 'model lumia 950', 'dtek60', 'moto maxx', 'liquid x2', 'galaxy j7 ( 2017 )', 'one max', 'a1', 'u ultra', 'pixel v2', 'mate 9 porsche design', 'liquid z520', 'one x10', 'galaxy c7 pro', 'x4 soul', 'galaxy c7 pro', 'redmi note prime', 'razr d1', 'zenfone 3 zoom ze553kl', 'moto maxx', 'f3 plus', 'galaxy c9 pro', 'r9s plus', 'f3', 'honor mx6', 'sure , v5s', 'xperia x performance', 'liquid jade primo', 'phone ms1', 'r11', 'k8 ( 2017 )', 'n3', 'galaxy c9 pro', 'v5s', 'xperia z4 tablet lte', '3t', 'galaxy j7 max', 'x9', 'v5s', 'a57', 'moto maxx', 'a57', 'galaxy c9 pro', 'x5max platinum edition', 'model xperia xa1 ultra', 'xperia xz premium', '2 3g', 'xperia xz premium', 'galaxy c7 pro', 'xperia xz', 'liquid jade primo', 'xperia l1', 'ms1', 'xperia xa1 ultra', 'mate 9', 'model desire 10 pro', 'xperia xz premium', 'mate 9', 'liquid x2', 'one e9+', 'one x10', 'marathon m5 plus', 'w909', 'zenfone live zb501kl', 'mate 9', 'axon lux', 'honor 7', 'galaxy a5 duos', 'a1', 'iconia talk s', 'model galaxy j3 emerge', 'liquid jade primo', 'note 5 lite', 'xperia xa ultra', 'xperia e4 dual', 'vibe p1m', 'xplay6', 'blade s7', 'x4 soul', 'zenfone 3s max zc521tl', 'pop 7 lte', 'dtek60', 'model nova 2', 'a59', 'r9 plus', 'model galaxy c9 pro', 'satio ( idou )', 'model mi note 2', 'galaxy j7 max', 'model galaxy c9 pro', 'galaxy s5 neo', 'model moto e3 power', 'droid turbo', 'model f1', 'model galaxy c7', 'r11 plus', 'signature touch ( 2015 )', 'then elife e8', 'mi 5c', 'blade a2 plus', 'galaxy j7 max', 'x7', 'a6600 plus', 'g6', 'm3 max', 'liquid x2', 'galaxy j3 ( 2017 )', 'one x10', 'hero cdma', 'moto x play dual sim', 'u11', '5', 'elife e8', 'zenfone 5 a500kl', 'iphone 7 plus', 'honor note 8', 'sure , g6', 'aurora', 's10', 's10', 'desire 830', 'galaxy s8', 'galaxy j7 ( 2016 )', 'v5 plus', 'redmi 3 pro', 'blade v8', 'axon 7 max', 'mi note 2', 'galaxy c9 pro', 'noir z8', 'xperia xa1 ultra', 'galaxy s7 active', "porsche design p'9982", '5', 'zenfone ar zs571kl', 'zenfone 3s max zc521tl', 'honor 9', 'nova 2', 'leap', 's10', 's9', 'pixel', 'moto e4 plus', 'eluga i3 mega', 'galaxy c7 pro', 'v3max', 'model y67', 'y67', 'idol 4s', 'blade a2 plus', 'a37', 'galaxy note7', 'galaxy a7 ( 2017 )', 'xperia c5 ultra dual', 'xperia xa dual', 'sure , model redmi note 4', 'mi 6', 'xplay5', 'z2', 'ms1', 'f1', 'f3 plus', 'liquid x2', 'zenfone 3 max zc553kl', 'lumia 630 dual sim', 'find 7a', 'galaxy s8+', 'galaxy s7', 'eluga ray x', 'xperia xz premium', 'mi note pro', 'iphone 6 plus', 'zenfone 3 laser zc551kl', 'z30', 'model butterfly 3', 'sure , f3', 'galaxy xcover 4', 'u ultra', 'one m9s', 'k6', 'g6', 'r7 plus', 'iconia talk s', 'x6s', 'blade qlux 4g', 'axon lux', 'noir e8', 'model zenfone live zb501kl', 'dtek50', 'nowadays s9', 'e7', 'model lumia 640 xl lte dual sim', 'model iphone 6s', 'sure , x5max platinum edition', 'model aurora', 'okay highway', 'ipaq hw6515', 'one m8s', 'desire 626g+', 'model honor 9', "model porsche design p'9983", 'r11', 'model moto e dual sim ( 2nd gen )', 'mc55', 'well , then v5s', 'model redmi 3 pro', 'p8 lite ( 2017 )', '6', 'galaxy xcover 4', 'sure , x9', 'moto e3 power', 'm2017', 'marathon m4', 'xperia z2a', 'zenfone 6 a601cg', 'aurora', 'iphone 7', 'galaxy a7 ( 2017 )', 'xperia e4g dual', 'xperia e5', 'xperia x', 'have a look at f5', 'sure , moto z2 play', 'iphone 7', 'conjr', 'fierce xl', 'p8 energy pro', 'v2 viper', 'blade v8', 'yutopia', '5 zs550kl', 'zenfone live zb501kl', 'a1 plus', 'cool s1', 'v10', 'r7 plus', 'gsmart classic lite', '1', 'k20 plus', 'mytouch q 2', 'canvas sliver 5 q450', 'f1 plus', 'xperia l1', '10 evo', 'honor 8 pro', 'e5', 'galaxy j2 prime', 'zenfone 2 deluxe ze551ml', 'marathon m5 plus', 'e6', 'eluga switch', 'm6s plus', 'sure , y7', 'sure , r3', 'moto z2 play', 'moto e4 plus ( usa )', 'canvas tab p666', 'p10 plus', 'le max 2', 'smart platinum 7', 'x7 plus', 'padfone infinity 2', 'a1 plus', 'x', 'eluga pulse', 'signature touch ( 2015 )', 'model v3', 'r9s', 'iphone 5s', 'pure xl', 'one ( m8 ) for windows', 'mate 9 pro', 'honor 3x pro', 'x10', 'model galaxy j1 ( 2016 )', 'galaxy c9 pro', 'honor magic', 'moto z force', 'light weight moto x ( 2nd gen )', 'one u11', 'iphone 7 plus', 'iphone 7', 'z30', 'zenfone go zb500kl', 'zenpad 3s 10 z500kl', '5 zs550kl', 'vivo iv', 'galaxy a7 ( 2017 )', 'lumia 2520', 'dtek50', 'galaxy c7', 'sure , w909', 'slate6 voicetab', 'model galaxy j7 prime', 'x venture', 'y28', 'blade s7', 'v5', 'tab prime 6', '3t', 'f3', 'one x10', 'x7', 'v3', 'xperia xa1 ultra', 'x6s', 'model xperia xa dual', 'xperia xa1', 'f3 plus', 'iphone 6s plus', 'model fire c', 'model zenfone ar zs571kl', 'priv', 'model iphone 7', 'galaxy s8', 'model s6 pro', 's9', 'model pioneer p6', 'model honor 8', 'model w909', 'blade v8', 'model blade g', 'iphone 6s', 'galaxy s8', 'model galaxy a5 ( 2017 )', 'x2 soul', 'mediapad m3 lite 10', '3', 's5015 spark ii', 'galaxy j3 emerge', 'galaxy note7', 'noir e8', 'galaxy xcover 4', 'mytouch q', 'canvas pulse 4g e451', 'pure xl', 's6', 'mi max 2', 'redmi 3 pro', 'redmi 3s', 'yutopia', 'pro 6 plus', 'iphone 5', '5 )', 'x3 soul pro', 'zenfone 5 a500kl', 'galaxy j7 max', 'galaxy j1 ace', 'galaxy j1 ace', 'galaxy s6 edge+', 'desire 728 dual sim', 'one m9s', 'a57', 'f1', 'eluga switch', 'pixel xl', 'one x10', 'z25', 'liquid z6 plus', 's9', 'honor 5a', 'model yoga tab 3 plus', 'u10', 'noir z8', 'xperia xa ultra', 'xp7', 'sl6010 cyprus lte', 'galaxy s8+']
ref_arr=['2012-','2012_','2020_','2020-','1212_','1212-'] 
quarter_to_month={'q4.released2009q4':'december','q1':'march','q2':'june','q3':'september','q4':'december','1q':'march','2q':'june','3q':'september','4q':'december'}

In [ ]:
vocab_f=open('data/glove/glove.6B.50d.txt',encoding='utf-8')
word_to_id={} 
word_to_emb={}
val=4
word_to_id['<pad>']=3 
word_to_id['<unk>']=0
word_to_id['<start>']=1
word_to_id['<end>']=2
for line in vocab_f:
  word_to_id[line.split()[0]]=val 
  word_to_emb[line.split()[0]]=line.split()[1:]  
  val+=1 
print(word_to_id['20']) 
print(len(word_to_emb['happy']))

328
50


In [ ]:
word_to_emb['<unk>']=[0]*50 
word_to_emb['<pad>']=[0]*50 
temp_agent=[word_to_emb['<unk>']]  
import pandas as pd
df=pd.read_excel('data/SharedData.xlsx') 
User=[] 
User_Arr=[]
for row in df['User']:
    if type(row)==int:
        User.append([]) 
    else:
        User[-1].append(row.lower()) 
        User_Arr.append(row.lower()) 
Agent=[]
Agent_Arr=[]
for row in df['Agent']:
    if type(row)==int:
        Agent.append([]) 
    else:
        Agent[-1].append(row.lower()) 
        Agent_Arr.append(row.lower()) 
Intent=[]
Intent_Arr=[]
for row in df['Intent']:
    if type(row)==int:
        Intent.append([]) 
    else:
        Intent[-1].append(row.lower()) 
        Intent_Arr.append(row.lower()) 
Slot=[]
Slot_Arr=[]
Value=[]
Value_Arr=[]
temp_slot=[]
temp_val=[]
is_slot=[]
cnt=0
for row in df['Slot']:
    if type(row)==int:
        Slot.append([]) 
        Value.append([]) 
        #temp_slot=[]
        #temp_val=[]
    elif type(row)==float:
        Slot[-1].append(['null']) 
        Value[-1].append(['null'])
        Slot_Arr.append(['null']) 
        Value_Arr.append(['null']) 
        temp_slot=['null'] 
        temp_val=['null']
    else: 
        temp_slot=[] 
        temp_val=[] 
        part=row.split(',') 
        Slot[-1].append([]) 
        Value[-1].append([])
        for p in part:
          Slot[-1][-1].append(correct[p.split('-')[0]]) 
          temp_slot.append(correct[p.split('-')[0]]) 
          if len(p.split('-'))>1:
            Value[-1][-1].append(val_mp[p.split('-')[1].lower()]) 
            temp_val.append(val_mp[p.split('-')[1].lower()])
            if temp_val[-1]=='t':
                temp_val[-1]='t-mobile'
          else:
            Value[-1][-1].append('null') 
            temp_val.append('null') 
    if type(row)!=int and type(row)!=float:
      Slot_Arr.append([]) 
      for a in temp_slot:
        Slot_Arr[-1].append(a)  
      Value_Arr.append([]) 
      for a in temp_val:
        Value_Arr[-1].append(a)    
A_Action=[]
A_Action_Arr=[]
for row in df['A_Action']:
    if type(row)==int:
        A_Action.append([])
    elif type(row)==float:
        A_Action[-1].append('null') 
    else:
        row=row.split(',')[0]
        A_Action[-1].append(a_action_mp[row]) 
        A_Action_Arr.append(a_action_mp[row]) 
U_Action=[]
U_Action_Arr=[]
for row in df['U_Action']:
    if type(row)==int:
        U_Action.append([])
    elif type(row)==float:
        U_Action[-1].append('null') 
        U_Action_Arr.append('null')  
    else:
        U_Action[-1].append(u_action_mp[row]) 
        U_Action_Arr.append(u_action_mp[row])

In [ ]:
word_to_emb['allview']=[str((float(word_to_emb['acer'][i])+float(word_to_emb['asus'][i]))/2) for i in range(50)]
word_to_emb['celkon']=[str((float(word_to_emb['zte'][i])+float(word_to_emb['meizu'][i]))/2) for i in range(50)]
word_to_emb['coolpad']=[str((float(word_to_emb['vivo'][i])+float(word_to_emb['oppo'][i]))/2) for i in range(50)]
word_to_emb['garmin-asus']=word_to_emb['asus']
word_to_emb['gionee']=[str((float(word_to_emb['xiaomi'][i])+float(word_to_emb['vivo'][i]))/2) for i in range(50)] 
word_to_emb['yezz']=[str((float(word_to_emb['zte'][i])+float(word_to_emb['pantech'][i]))/2) for i in range(50)]
word_to_emb['i-mate']=[str((float(word_to_emb['kyocera'][i])+float(word_to_emb['jolla'][i]))/2) for i in range(50)]
word_to_emb['intex']=[str((float(word_to_emb['micromax'][i])+float(word_to_emb['lava'][i]))/2) for i in range(50)]
word_to_emb['leeco']=[str((float(word_to_emb['intex'][i])+float(word_to_emb['i-mate'][i]))/2) for i in range(50)]
word_to_emb['oneplus']=[str((float(word_to_emb['apple'][i])+float(word_to_emb['samsung'][i]))/2) for i in range(50)]
word_to_emb['karbonn']=word_to_emb['karbon']
word_to_emb['prestigio']=[str((float(word_to_emb['apple'][i])+float(word_to_emb['asus'][i]))/2) for i in range(50)]
word_to_emb['qmobile']=[str((float(word_to_emb['nokia'][i])+float(word_to_emb['oppo'][i]))/2) for i in range(50)]
word_to_emb['verykool']=[str((float(word_to_emb['coolpad'][i])+float(word_to_emb['lg'][i]))/2) for i in range(50)]
word_to_emb['wiko']=[str((float(word_to_emb['oppo'][i])+float(word_to_emb['asus'][i]))/2) for i in range(50)]
word_to_emb['sonim']=[str((float(word_to_emb['sony'][i])+float(word_to_emb['samsung'][i]))/2) for i in range(50)]
word_to_emb['sony ericsson']=word_to_emb['ericsson'] 

In [ ]:
inp_encoder=[]
mx_encoder_len=0 
for i in range(500):
  for j in range(len(User[i])):
    inp_encoder.append([])
    if j==0:
      bspans=[]
      for k in range(15):
        inp_encoder[-1].append([0]*50) 
    else:
      #bspans adding 
      if Slot[i][j][0]!='null':
        for k in range(len(Slot[i][j-1])):
          bspans.append([Slot[i][j-1][k],Value[i][j-1][k]])
      sent=padd(Agent[i][j-1])
      for k in range(15):
        if sent[k] in word_to_emb:
          inp_encoder[-1].append(word_to_emb[sent[k]])
        else:
          inp_encoder[-1].append(word_to_emb['<unk>']) 
    
    sent=padd(User[i][j]) 
    for k in range(15):
      if sent[k] in word_to_emb:
        inp_encoder[-1].append(word_to_emb[sent[k]]) 
      else:
        inp_encoder[-1].append(word_to_emb['place']) 
    for k in range(len(bspans)):
      for l in range(2):
        if bspans[k][l] in word_to_emb:
          inp_encoder[-1].append(word_to_emb[bspans[k][l]]) 
        else:
          inp_encoder[-1].append(word_to_emb['place'])  
    mx_encoder_len=max(len(inp_encoder[-1]),mx_encoder_len)  
for i in range(len(inp_encoder)):
  inp_len=len(inp_encoder[i])
  for j in range(40-inp_len):
    inp_encoder[i].append(word_to_emb['<unk>']) 

In [ ]:
out_encoder=[]
out_encoder_len=0 
for i in range(500):
  for j in range(len(User[i])):
    out_encoder.append([])
    if j==0:
      bspans=[] 
      #bspans adding 
    if Slot[i][j][0]!='null':
      for k in range(len(Slot[i][j])):
        bspans.append([Slot[i][j][k],Value[i][j][k]]) 
    for k in range(len(bspans)):
      for l in range(2):
        if bspans[k][l] in word_to_emb:
          out_encoder[-1].append(word_to_emb[bspans[k][l]]) 
        else:
          out_encoder[-1].append(word_to_emb['place'])  
    out_encoder_len=max(len(out_encoder[-1]),out_encoder_len) 
for i in range(len(out_encoder)):
  out_len=len(out_encoder[i]) 
  for j in range(12-out_len): 
    out_encoder[i].append(word_to_emb['<unk>']) 
  out_encoder[i]=out_encoder[i][:12] 

In [ ]:
len(out_encoder)

4201

In [ ]:
def encoder_model(input,output):
  model = Sequential()
  model.add(LSTM(100, activation='relu', input_shape=(40,50)))
  model.add(RepeatVector(12))
  model.add(LSTM(100, activation='relu', return_sequences=True))
  model.add(TimeDistributed(Dense(50)))
  model.compile(optimizer='adam', loss='mse')
  # fit model
  model.fit(input, output, epochs=3, verbose=0) 
  return model

In [ ]:
inp_encoder_np=np.array(inp_encoder).reshape(4201,40,50) 
out_encoder_np=np.array(out_encoder).reshape(4201,12,50) 
model=encoder_model(inp_encoder_np,out_encoder_np)  

In [ ]:
inp_temp=model.predict(inp_encoder_np[:10]) 
print(inp_temp[0])

In [ ]:
mp_sl,mp_vl={},{} 
value,slot=[],[] 
cnt_val,cnt_slt=0,0
for i in range(500):
  for j in range(len(Slot[i])):
    for k in range(len(Slot[i][j])):
      slt,val=Slot[i][j][k],Value[i][j][k]
      if slt not in mp_sl:
        mp_sl[slt]=cnt_slt 
        cnt_slt+=1 
      if val not in mp_vl:
        mp_vl[val]=cnt_val 
        cnt_val+=1 
slot_arr=[] 
for i in range(500):
  for j in range(len(Slot[i])):
    slot_arr.append([])  
    for k in range(cnt_slt):
      slot_arr[-1].append(0) 
    for slt in Slot[i][j]:
      slot_arr[-1][mp_sl[slt]]=1  
print(len(slot_arr))
print(slot_arr[996])

4201
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
inp_slot=[] 
out_slot=[]
for i in range(4201):
  for j in range(len(Slot_Arr[i])):
    out_slot.append([])
    inp_slot.append([]) 
    for k in range(40):
      inp_slot[-1].append(inp_encoder[i][k]) 
    for k in range(25):
      if mp_sl[Slot_Arr[i][j]]==k:
        out_slot[-1].append(1) 
      else:
        out_slot[-1].append(0) 

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, 50))
encoder = LSTM(50, return_state=True)  
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
enc_dense=Dense(50,activation='relu') 
enc_out=enc_dense(encoder_outputs)
encoder_model = Model(encoder_inputs,encoder_outputs) 
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, 50))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=[enc_out,enc_out])
decoder_dense = Dense(25, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
slot_arr_np=np.array(slot_arr).reshape(4201,1,25)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([inp_encoder_np, out_encoder_np], slot_arr_np,
          epochs=2,
          validation_split=0.2)

Train on 3360 samples, validate on 841 samples
Epoch 1/2
3360/3360 [==============================] - 10s 3ms/step - loss: 1.7696 - accuracy: 0.7291 - val_loss: 1.3808 - val_accuracy: 0.7353
Epoch 2/2
3360/3360 [==============================] - 9s 3ms/step - loss: 1.2027 - accuracy: 0.7697 - val_loss: 1.1594 - val_accuracy: 0.7968
